In [7]:
import os
import gc
import heapq
import pickle
import numba as nb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import math


In [8]:
%%time
df = pd.read_csv("../../allData/validationData/train_meta_data.csv")
df_test = pd.read_csv("../../allData/validationData/test_meta_data.csv")
df = pd.concat([df, df_test]).reset_index(drop = True)
npz = np.load("../../allData/validationData/train_core_data.npz")
npz_test = np.load("../../allData/validationData/test_core_data.npz")
aids = np.concatenate([npz['aids'], npz_test['aids']])
ts = np.concatenate([npz['ts'], npz_test['ts']])
ops = np.concatenate([npz['ops'], npz_test['ops']])

df["start_idx"] = df['total_action'].cumsum().shift(1).fillna(0).astype(int)
df["end_time"] = ts[df["start_idx"] + df["total_action"] - 1]

CPU times: user 6.77 s, sys: 3.38 s, total: 10.2 s
Wall time: 11.1 s


In [9]:
df

,session,total_action,session_start_time,session_end_time,start_idx,end_time
0,0,147,1659304800,1661103727,0,1661103727
1,1,27,1659304800,1660857067,147,1660857067
2,2,13,1659304800,1660577379,174,1660577379
3,3,226,1659304800,1661109666,187,1661109666
4,4,3,1659304800,1659304900,413,1659304900
...,...,...,...,...,...,...
12368249,12899774,1,1661723968,1661723968,171022132,1661723968
12368250,12899775,1,1661723970,1661723970,171022133,1661723970
12368251,12899776,1,1661723972,1661723972,171022134,1661723972
12368252,12899777,1,1661723976,1661723976,171022135,1661723976


In [10]:
## Define constants
PARALLEL = 1024
LOOKBACK_WINDOW = 100   ## only fit the latest LOOKBACK_WINDOW to train the sim matrix
TOPN = 20
ACTION_WEIGHTS = np.array([1.0, 2.0, 1.0])

## Training

In [11]:
@nb.jit(nopython=True)
def getItemTotalLikes(aids, ops, item_total_likes, action_weights):
    """
    Stores the total like score of itemXXX in item_total_likes, based on action_weights parameter. np.array([X, Y, Z])
    """
    for idx, item in enumerate(aids):
        if item not in item_total_likes: 
            item_total_likes[item] = 0
        item_total_likes[item] += action_weights[ops[idx]]   ## TODO: For time decay, consider replace with 1, for iuf keep this. 
        

@nb.jit(nopython=True)
def getSimScoresSingleRow(pairs_this_row, start_time, start_idx, length, aids, ts, ops, action_weights, mode):
    """
    Get the sim scores of items within single session, can be ran in parallel within each batch. 
    """
    max_idx = start_idx + length
    min_idx = max(max_idx - LOOKBACK_WINDOW, start_idx)  
    for i in range(min_idx, max_idx):
        for j in range(i+1, max_idx):
            if ts[j] - ts[i] > 2 * 60 * 60: continue  #TODO: try 2h only
            if aids[i] == aids[j]: continue
            
            if mode == "cosine":
                w_ij = action_weights[ops[j]] 
                w_ji = action_weights[ops[i]] 
            elif mode == "iuf":  ## penalize users that had lots of actions TODO: consider location weight
                if ts[max_idx] - start_time > 24 * 60 * 60:
                    special_factor = 0.8
                else:
                    special_factor = 1.0
                
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))
                w_ij = special_factor * action_weights[ops[j]] * time_gap_weight * loc_weight / math.log1p(length)
                w_ji = special_factor * action_weights[ops[i]] * time_gap_weight * loc_weight / math.log1p(length)
            elif mode == "time_decay":
                ## calculate some time weights of each item, more weights are given when ts is later. #TODO: try adding (i-j) location weight, exponential weight, 0.5 ** (abs(i-j + 1)), 
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                #time_i = 1 + 0.1 ** ((1662328791-ts[i])/(1662328791-1659304800)) #1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800) #  #(1 - 0.8 *(TEST_END_TS - ts[i]) / TIME_SPAN) ** 0.5 # 0.2~1 #   ## time decay weight for item i 
                #time_j = 1 + 0.1 ** ((1662328791-ts[j])/(1662328791-1659304800))  # 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) # #  #(1 - 0.8 *(TEST_END_TS - ts[j]) / TIME_SPAN) ** 0.5   # 
                time_i = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[i])/(1662328791-1659304800)) - 0.6  )))
                time_j = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[j])/(1662328791-1659304800)) - 0.6  )))
                
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))
                
                w_ij = action_weights[ops[j]] * loc_weight * time_gap_weight * time_i / math.log1p(length)
                w_ji = action_weights[ops[i]] * loc_weight * time_gap_weight * time_j / math.log1p(length)
                
            pairs_this_row[(aids[i], aids[j])] = w_ij
            pairs_this_row[(aids[j], aids[i])] = w_ji

@nb.jit(nopython=True, parallel=True, cache=True)
def getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, action_weights, total_weights, mode="cosine"):
    nrows = len(rows)
    pairs_this_batch = [{(0, 0): 0.0 for _ in range(0)} for _ in range(nrows)]
    ## get the sim scores of each batch in seperate sub dict in pairs_this_batch
    for row_i in nb.prange(nrows):  ## run each row of the batch in parallel
        _, start_idx, length, start_time = rows[row_i]
        getSimScoresSingleRow(pairs_this_batch[row_i], start_time, start_idx, length, aids, ts, ops, action_weights, mode)
    ## merge pairs_this_batch into the fullSimMatrix
    for row_i in range(nrows):
        for (aid1, aid2), score in pairs_this_batch[row_i].items():
            if aid1 not in fullSimMatrix: 
                fullSimMatrix[aid1] = {0: 0.0 for _ in range(0)}
            if aid2 not in fullSimMatrix[aid1]:
                fullSimMatrix[aid1][aid2] = 0.0
            fullSimMatrix[aid1][aid2] += score
    

@nb.jit(nopython = True)
def heap_topk(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure
    """
    dic = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q) for _ in range(len(q))][::-1]
    for i in range(len(res)):
        dic[res[i][1]] = res[i][0]
    
    return dic
   
@nb.jit(nopython = True)
def trim_simMatrix_topk(fullSimMatrix, k = 50):
    """
    trim top k items of each "itemX: {itemY: score1, ...}" pair in fullSimMatrix based on sim scores. 
    """
    for item, item_cnt_dict in fullSimMatrix.items():
        fullSimMatrix[item] = heap_topk(item_cnt_dict, k)

In [12]:
@nb.jit(nopython=True)
def itemTotalLikeNorm(fullSimMatrix, item_total_likes):
    for aid_1, relations in fullSimMatrix.items():
        for aid_2, sim_score in relations.items():
            fullSimMatrix[aid_1][aid_2] = sim_score / (item_total_likes[aid_1] * item_total_likes[aid_2]) ** 0.1  ## TODO: consider 0.1 or other small number
            
@nb.jit(nopython=True)
def maxNormSimMatrix(fullSimMatrix):
    for aid_1, relations in fullSimMatrix.items():
        max_num = -np.inf
        for _, sim_score in relations.items():
            if sim_score > max_num:
                max_num = sim_score
        ## DEGUG use, delete later
        if max_num == 0:
            print(aid_1)
            print(fullSimMatrix[aid_1])
        for aid_2, sim_score in relations.items():
#             if max_num == 0:
#                 max_num += 0.001
            fullSimMatrix[aid_1][aid_2] = sim_score / max_num

In [13]:
%%time
## get the Total Like matrix
item_total_likes = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.float64)

getItemTotalLikes(aids, ops, item_total_likes, ACTION_WEIGHTS)

CPU times: user 18.7 s, sys: 782 ms, total: 19.5 s
Wall time: 20 s


In [8]:
%%time
simMatrices = {}   ## store a few different similarity matrices using different scoring system, for different prediction type
TRIM_CYCLES = 1000   ## trim full sim matrix every XX batches. 
MODES_TO_TRAIN = ["iuf", "time_decay"]

for mode in MODES_TO_TRAIN:
    ## the nested dict to store full sim matrix, {itemX: {itemY: score, itemZ: score, ...}}
    fullSimMatrix = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
    max_idx = len(df)
    batch_idx = 1  ## compute sim matrix for PARALLEL # of rows per batch, have a total of max_idx/PARALLEL batches.
    for idx in tqdm(range(0, max_idx, PARALLEL)):
        rows = df.iloc[idx: min(idx + PARALLEL, max_idx)][['session', 'start_idx', 'total_action', 'session_start_time']].values
        getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, ACTION_WEIGHTS, item_total_likes, mode=mode)
        batch_idx += 1
        if batch_idx % TRIM_CYCLES == 0:
            print("batch_idx: ", batch_idx)
            trim_simMatrix_topk(fullSimMatrix, 100)
            gc.collect()
            #break

    
    ## trim top 50 when the training is complete
    trim_simMatrix_topk(fullSimMatrix, 80)   ## TODO: make this num small enough to reduce time for normalization
    ## normalize the fullSimMatrix with item popularity
    itemTotalLikeNorm(fullSimMatrix, item_total_likes)
    ## max norm of each score
    maxNormSimMatrix(fullSimMatrix)
    
    simMatrices[mode] = fullSimMatrix
    
    del fullSimMatrix
    gc.collect()

  8%|▊         | 998/12079 [01:57<33:03,  5.59it/s] 

batch_idx:  1000


 17%|█▋        | 1999/12079 [04:54<28:36:17, 10.22s/it]

batch_idx:  2000


 25%|██▍       | 3000/12079 [07:01<10:26:13,  4.14s/it]

...

batch_idx:  11000


 99%|█████████▉| 11994/12079 [25:07<00:02, 41.16it/s]  

batch_idx:  12000


100%|██████████| 12079/12079 [25:56<00:00,  7.76it/s]

CPU times: user 1h 25min 19s, sys: 49min 18s, total: 2h 14min 38s
Wall time: 52min 16s


In [2]:
simMatrices["iuf"][1517085]

NameError: name 'simMatrices' is not defined

In [21]:
aids

array([1517085, 1563459, 1309446, ...,  548599,  384045,  561560])

## Inference

In [10]:
@nb.jit(nopython = True)
def heap_topk_return_list(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure, return a list with top "cap" elements with highest score
    """
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q)[1] for _ in range(len(q))][::-1]
    
    return res

@nb.jit(nopython=True)
def getRealTimeActionWeight(ts_action, ts_start, ts_end, op, seq, length, ACTION_WEIGHTS, TRAIN_START_TS, TEST_END_TS):
    """
    This function returns the real time importance weight of test set session actions
    input: 
        ts_action: ts of the action takes place
        ts_start: start_ts of this session
        ts_end: end_ts of this session
        op: type of the action
        seq: seq order this action
        length: total # of actions of this session. 
        
    CONSTANTS: 
    ACTION_WEIGHTS
    TRAIN_START_TS
    TEST_END_TS
    """
#     overall_time_span = TEST_END_TS - TRAIN_START_TS
#     #session_time_span = ts_end - ts_start
#     time_to_end_of_session = ts_end - ts_action
#     if time_to_end_of_session > 24 * 60 * 60:
#         time_weight = 0.3
#     else:
#         time_weight = max(2**(1 - time_to_end_of_session/3600) - 1, 0.4)
    
    ## TODO: add time decay 
    
    action_weight = ACTION_WEIGHTS[op]
    
    sequence_weight = max(2 ** (seq/length) - 1, 0.5) ## floor at 0.1  #np.power(2, np.linspace(seq_weight_const, 1, length))[::-1] - 1

    res = action_weight * sequence_weight # * time_weight
    
    return res

@nb.jit(nopython=True)
def inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    ending_idx = starting_idx + length
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]
    
    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
    
    if length >= 20:   ## if the user has many actions, recommend based on the itemes it had been interacted with only.
        sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
        for aid, op, w in zip(candidates, candidates_ops, sequence_weight):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            potential_to_recommend[aid] += w * test_ops_weights[op]
        result[session] = np.array(heap_topk_return_list(potential_to_recommend, 20)) 
    else:   ## other wise recommend based on the similar items.
        for idx in range(starting_idx, ending_idx):
            candidate = aids[idx]
            if candidate not in potential_to_recommend:
                potential_to_recommend[candidate] = np.inf ## ensure large weights on items had interacted with. 
        
        sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates))) - 1
        for idx in range(starting_idx, ending_idx):
            candidate = aids[idx] 
            time_weight = 1 + 0.1 ** ((1662328791-ts[idx])/(1662328791-1659304800))
            candidate_realtime_weight = test_ops_weights[ops[idx]] * sequence_weight[idx-starting_idx] * time_weight  #getRealTimeActionWeight(ts[idx], ts[starting_idx], ts[ending_idx], ops[idx], idx-starting_idx+1, length, ACTION_WEIGHTS, TRAIN_START_TS, TEST_END_TS)
            ## load the potential items to recommend,
            if candidate not in full_sim_matrix: 
                continue
            for similar_item in full_sim_matrix[candidate]:
#                 if similar_item in candidates:    ## skip the item if the it's already been interacted
#                     continue
                if similar_item not in potential_to_recommend:
                    potential_to_recommend[similar_item] = 0
                potential_to_recommend[similar_item] += full_sim_matrix[candidate][similar_item] * candidate_realtime_weight 
        result[session] = np.array(heap_topk_return_list(potential_to_recommend, 20))
    
@nb.jit(nopython=True)
def run_inference_parallel(rows, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights)

In [11]:
%%time
result_iuf = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

result_time_decay = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
    run_inference_parallel(rows, aids, ops, ts, result_iuf, simMatrices["iuf"], ACTION_WEIGHTS)
    run_inference_parallel(rows, aids, ops, ts, result_time_decay, simMatrices["time_decay"], ACTION_WEIGHTS)

100%|██████████| 1742/1742 [04:36<00:00,  6.31it/s]

CPU times: user 3min 8s, sys: 50 s, total: 3min 58s
Wall time: 4min 36s


In [16]:
%%time
subs = []
op_names = ["clicks", "carts", "orders"]

for result, op in zip([result_time_decay, result_iuf, result_iuf], op_names):
    sub = pd.DataFrame({"session_type": result.keys(), "labels": result.values()})
    sub.session_type = sub.session_type.astype(str) + f"_{op}"
    sub.labels = sub.labels.apply(lambda x: " ".join(x.astype(str)))
    subs.append(sub)
    
submission = pd.concat(subs).reset_index(drop=True)
#sub.sort_values(by=["session_type"])  ## optional
#submission.to_csv('submission.csv', index = False)

CPU times: user 1min 44s, sys: 4.53 s, total: 1min 49s
Wall time: 1min 53s


In [17]:
%%time
submission.to_csv('../../allData/validationData/submission_time_dec_onCarts.csv', index = False)